In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import pandas_profiling
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
pd.set_option("display.max_columns", 999)
pd.set_option('max_colwidth', 25)
pd.set_option('colheader_justify', 'left')

In [3]:
df_london = pd.read_csv("/Users/valmadrid/DataScienceBootcamp/Projects/Mod4/London Airbnb Listing Detailed.csv")
df_london.head(3)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,11551.0,https://www.airbnb.co...,2.019091e+13,2019-09-15,Arty and Bright Londo...,Unlike most rental ap...,Amenities Bedding: 1 ...,Unlike most rental ap...,family,Not even 10 minutes b...,No Smoking (very stri...,Tons of buses (24hrs)...,Guest will have acces...,No interaction with g...,No Smoking (very stri...,NaN,NaN,https://a0.muscache.c...,NaN,43039,https://www.airbnb.co...,Adriano,2009-10-03,"London, England, Unit...","Hello, I'm a friendly...",within an hour,100%,NaN,t,https://a0.muscache.c...,https://a0.muscache.c...,Brixton,3.0,3.0,"['email', 'phone', 'r...",t,t,"London, United Kingdom",Brixton,Lambeth,NaN,London,NaN,SW9 8DG,London,"London, United Kingdom",GB,United Kingdom,51.46225,-0.11732,t,Apartment,Entire home/apt,4.0,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Intern...",NaN,$88.00,$645.00,"$2,350.00",$400.00,$30.00,2.0,$20.00,2.0,180.0,2,2.0,180.0,180,2.0,180.0,4 days ago,t,19.0,42.0,72.0,344.0,2019-09-15,184.0,1.0,2010-03-21,2019-07-10,92.0,9.0,9.0,10.0,10.0,9.0,9.0,f,NaN,NaN,t,f,strict_14_with_grace_...,f,t,2.0,2.0,0.0,0.0,1.59
1,13913.0,https://www.airbnb.co...,2.019091e+13,2019-09-15,Holiday London DB Roo...,My bright double bedr...,"Hello Everyone, I'm o...",My bright double bedr...,business,Finsbury Park is a fr...,For art lovers I can ...,The flat only a 10 mi...,Guest will have acces...,I like to have little...,I'm an artist and hav...,NaN,NaN,https://a0.muscache.c...,NaN,54730,https://www.airbnb.co...,Alina,2009-11-16,"London, England, Unit...",I am a Multi-Media Vi...,within an hour,100%,NaN,f,https://a0.muscache.c...,https://a0.muscache.c...,LB of Islington,4.0,4.0,"['email', 'phone', 'f...",t,f,"Islington, Greater Lo...",LB of Islington,Islington,NaN,Islington,Greater London,N4 3,London,"Islington, United Kin...",GB,United Kingdom,51.56802,-0.11121,t,Apartment,Private room,2.0,1.0,1.0,0.0,Real Bed,"{TV,""Cable TV"",Wifi,K...",538.0,$65.00,$333.00,"$1,176.00",$100.00,$15.00,1.0,$15.00,1.0,29.0,1,1.0,29.0,29,1.0,29.0,5 months ago,t,18.0,48.0,78.0,353.0,2019-09-15,16.0,2.0,2010-08-18,2019-06-10,96.0,9.0,10.0,9.0,10.0,9.0,9.0,f,NaN,NaN,f,f,moderate,f,f,3.0,1.0,2.0,0.0,0.14
2,15400.0,https://www.airbnb.co...,2.019091e+13,2019-09-15,Bright Chelsea Apart...,Lots of windows and l...,Bright Chelsea Apartm...

In [5]:
df_london.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85274 entries, 0 to 85273
Columns: 106 entries, id to reviews_per_month
dtypes: float64(35), object(71)
memory usage: 69.0+ MB


In [4]:
df_london.select_dtypes("object").columns

Index(['listing_url', 'last_scraped', 'name', 'summary', 'space',
       'description', 'experiences_offered', 'neighborhood_overview', 'notes',
       'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url',
       'picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'is_location_exact',
       'property_type', 'room_type', 'bed_type', 'amenities', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'minimum

In [6]:
df_london.select_dtypes("float64").columns

Index(['id', 'scrape_id', 'medium_url', 'xl_picture_url',
       'host_listings_count', 'latitude', 'longitude', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'square_feet', 'guests_included',
       'minimum_nights', 'maximum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object')

In [8]:
col = list(df_london.columns)
col

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',


In [10]:
columns_for_deletion = ["listing_url", "scrape_id", "name", "summary", "space", "description", "neighborhood_overview", "notes", "transit", "access", "interaction", "house_rules", 
           "thumbnail_url", "medium_url", "picture_url", "xl_picture_url", "host_id", "host_url", "host_name", "host_location", "host_about", "host_response_time", 
           "host_response_rate", "host_acceptance_rate", "host_thumbnail_url", "host_picture_url", "host_neighbourhood", "host_listings_count", 
           "host_total_listings_count", "street", "neighbourhood_group_cleansed", "city", "state", "zipcode", "market", "smart_location", "country_code", "country", 
           "square_feet", "minimum_minimum_nights", "maximum_minimum_nights", "minimum_maximum_nights", "maximum_maximum_nights", "minimum_nights_avg_ntm", 
           "maximum_nights_avg_ntm", "calendar_updated", "has_availability", "calendar_last_scraped", "license", "jurisdiction_names", "is_business_travel_ready"]


In [17]:
diff = list(set(col).difference(set(columns_for_deletion)))

In [18]:
diff.sort()
diff

['accommodates',
 'amenities',
 'availability_30',
 'availability_365',
 'availability_60',
 'availability_90',
 'bathrooms',
 'bed_type',
 'bedrooms',
 'beds',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms',
 'cancellation_policy',
 'cleaning_fee',
 'experiences_offered',
 'extra_people',
 'first_review',
 'guests_included',
 'host_has_profile_pic',
 'host_identity_verified',
 'host_is_superhost',
 'host_since',
 'host_verifications',
 'id',
 'instant_bookable',
 'is_location_exact',
 'last_review',
 'last_scraped',
 'latitude',
 'longitude',
 'maximum_nights',
 'minimum_nights',
 'monthly_price',
 'neighbourhood',
 'neighbourhood_cleansed',
 'number_of_reviews',
 'number_of_reviews_ltm',
 'price',
 'property_type',
 'require_guest_phone_verification',
 'require_guest_profile_picture',
 'requires_license',
 'review_scores_accuracy',
 'review_scores_checki

In [4]:
df_london.amenities.iloc[1]

'{TV,"Cable TV",Wifi,Kitchen,"Paid parking off premises","Smoking allowed","Free street parking","Buzzer/wireless intercom",Heating,"Family/kid friendly",Washer,Dryer,"Smoke detector","Carbon monoxide detector","Fire extinguisher",Essentials,Shampoo,"Lock on bedroom door",Hangers,"Hair dryer",Iron,"Laptop friendly workspace","Outlet covers",Bathtub,"Children’s books and toys","Babysitter recommendations",Crib,"Pack ’n Play/travel crib","Room-darkening shades","Children’s dinnerware","Hot water","Bed linens","Extra pillows and blankets","Ethernet connection","Coffee maker",Refrigerator,"Dishes and silverware","Cooking basics",Oven,Stove,"Patio or balcony","Luggage dropoff allowed","Long term stays allowed","No stairs or steps to enter","Wide entrance","Extra space around bed","Accessible-height bed","No stairs or steps to enter","Wide doorway to guest bathroom","Bathtub with bath chair","Accessible-height toilet","Host greets you","Handheld shower head","Roll-in shower"}'

In [6]:
df_londonairbnb = pd.read_csv("londonairbnb_cleaned.csv")
df_londonairbnb

,id,last_scraped,experiences_offered,host_since,host_is_superhost,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_days,days_from_last_review,amenities_count
0,11551.0,2019-09-15,family,2009-10-03,1,"['email', 'phone', 'r...",1,1,Brixton,Lambeth,51.46225,-0.11732,1,Apartment,Entire home/apt,4.0,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Intern...",88.0,645.0,2350.0,400.0,30.0,2.0,20.0,2.0,180.0,19.0,42.0,72.0,344.0,184.0,1.0,2010-03-21,2019-07-10,92.0,9.0,9.0,10.0,10.0,9.0,9.0,0,1,strict_14_with_grace_...,0,1,2.0,2.0,0.0,0.0,1.59,3634.0,67.0,23
1,13913.0,2019-09-15,business,2009-11-16,0,"['email', 'phone', 'f...",1,0,LB of Islington,Islington,51.56802,-0.11121,1,Apartment,Private room,2.0,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Wifi,K...",65.0,333.0,1176.0,100.0,15.0,1.0,15.0,1.0,29.0,18.0,48.0,78.0,353.0,16.0,2.0,2010-08-18,2019-06-10,96.0,9.0,10.0,9.0,10.0,9.0,9.0,0,0,moderate,0,0,3.0,1.0,2.0,0.0,0.14,3590.0,97.0,55
2,15400.0,2019-09-15,romantic,2009-12-05,1,"['email', 'phone', 'r...",1,1,Chelsea,Kensington and Chelsea,51.48796,-0.16898,1,Apartment,Entire home/apt,2.0,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Intern...",100.0,600.0,2250.0,150.0,50.0,2.0,0.0,30.0,50.0,8.0,8.0,8.0,114.0,87.0,6.0,2009-12-21,2019-08-27,96.0,10.0,10.0,10.0,10.0,10.0,9.0,0,1,strict_14_with_grace_...,1,1,1.0,1.0,0.0,0.0,0.73,3571.0,19.0,29
3,17402.0,2019-09-15,none,2010-01-04,1,"['email', 'phone', 'r...",1,1,City of Westminster,Westminster,51.52098,-0.14002,1,Apartment,Entire home/apt,6.0,2.0,3.0,4.0,Real Bed,"{TV,Wifi,Kitchen,""Pai...",300.0,1378.0,0.0,350.0,65.0,4.0,10.0,3.0,365.0,12.0,36.0,54.0,265.0,41.0,5.0,2011-03-21,2019-06-19,94.0,10.0,9.0,9.0,9.0,10.0,9.0,0,1,strict_14_with_grace_...,0,0,15.0,15.0,0.0,0.0,0.40,3541.0,88.0,31
4,24328.0,2019-09-15,family,2009-09-28,0,"['email', 'phone', 'r...",1,1,LB of Wandsworth,Wandsworth,51.47298,-0.16376,1,Townhouse,Entire home/apt,2.0,1.5,1.0,1.0,Real Bed,"{TV,""Cable TV"",Intern...",175.0,1050.0,3500.0,250.0,70.0,2.0,0.0,30.0,1125.0,11.0,41.0,71.0,346.0,93.0,1.0,2010-11-15,2019-06-08,98.0,10.0,10.0,10.0,10.0,9.0,9.0,0,0,moderate,1,1,1.0,1.0,0.0,0.0,0.86,3639.0,99.0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85251,38612345.0,2019-09-15,none,2018-07-27,0,"['email', 'phone', 'o...",1,0,LB of Ealing,Ealing,51.51383,-0.27288,0,Apartment,Entire home/apt,5.0,1.5,1.0,3.0,Real Bed,"{TV,Wifi,Kitchen,""Fre...",85.0,0.0,0.0,0.0,40.0,1.0,0.0,2.0,1125.0,7.0,20.0,38.0,45.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,flexible,0,0,7.0,7.0,0.0,0.0,0.00,415.0,0.0,15
85252,38612360.0,2019-09-15,none,2017-09-12,0,"['email', 'phone', 'j...",1,0,Hampstead,Camden,51.55569,-0.21251,0,Apartment,Private room,2.0,2.0,1.0,1.0,Real Bed,"{Kitchen,Heating,Wash...",35.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1125.0,2.0,2.0,2.0,2.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,flexible,0,0,2.0,0.0,2.0,0.0,0.00,733.0,0.0,16
85253,38613001.0,2019-09-15,none,2019-09-14,0,"['email', 'phone

In [8]:
#Model no. 1

X_columns_exlude = ['id', 'last_scraped', 'host_since', 'amenities', 'host_verifications', 'neighbourhood', 'latitude', 'longitude', 'weekly_price',
                   'monthly_price', 'availability_30', 'availability_60', 'availability_90', 'first_review','last_review', 'price',
                    'calculated_host_listings_count', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
                    'review_scores_communication', 'review_scores_location','review_scores_value']

X = df_londonairbnb.drop(columns = X_columns_exlude, axis=1)
y = df_londonairbnb[["price"]]

model1 = pd.concat([y,X], axis = 1)
model1.head()

,price,experiences_offered,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,requires_license,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_days,days_from_last_review,amenities_count
0,88.0,family,1,1,1,Lambeth,1,Apartment,Entire home/apt,4.0,1.0,1.0,1.0,Real Bed,400.0,30.0,2.0,20.0,2.0,180.0,344.0,184.0,1.0,92.0,0,1,strict_14_with_grace_...,0,1,2.0,0.0,0.0,1.59,3634.0,67.0,23
1,65.0,business,0,1,0,Islington,1,Apartment,Private room,2.0,1.0,1.0,1.0,Real Bed,100.0,15.0,1.0,15.0,1.0,29.0,353.0,16.0,2.0,96.0,0,0,moderate,0,0,1.0,2.0,0.0,0.14,3590.0,97.0,55
2,100.0,romantic,1,1,1,Kensington and Chelsea,1,Apartment,Entire home/apt,2.0,1.0,1.0,1.0,Real Bed,150.0,50.0,2.0,0.0,30.0,50.0,114.0,87.0,6.0,96.0,0,1,strict_14_with_grace_...,1,1,1.0,0.0,0.0,0.73,3571.0,19.0,29
3,300.0,none,1,1,1,Westminster,1,Apartment,Entire home/apt,6.0,2.0,3.0,4.0,Real Bed,350.0,65.0,4.0,10.0,3.0,365.0,265.0,41.0,5.0,94.0,0,1,strict_14_with_grace_...,0,0,15.0,0.0,0.0,0.40,3541.0,88.0,31
4,175.0,family,0,1,1,Wandsworth,1,Townhouse,Entire home/apt,2.0,1.5,1.0,1.0,Real Bed,250.0,70.0,2.0,0.0,30.0,1125.0,346.0,93.0,1.0,98.0,0,0,moderate,1,1,1.0,0.0,0.0,0.86,3639.0,99.0,27


In [15]:
import regression as reg
%reload_ext autoreload
%autoreload 2

In [10]:
X_train, X_test, y_train, y_test = reg.preprocess(X, y)

In [11]:
#First Linear Regression Model

train_R2, test_R2, train_mse, test_mse, coefs, intercept = reg.run_linear(X_train, X_test, y_train, y_test)

print('Training r^2:', train_R2)
print('Testing r^2:', test_R2)
print('Training MSE:', train_mse)
print('Testing MSE:', test_mse)

Training r^2: 0.06262437866645554
Testing r^2: -17838719600184.61
Training MSE: 188971.74569330874
Testing MSE: 3.350304240622576e+18


In [16]:
#First LASSO model

lasso_table = reg.run_lasso(X_train, X_test, y_train, y_test)
lasso_table.head()

,alpha,Training_r^2,MSE_train,Testing_r^2,MSE_test,Intercept,Coefficients
0,0.000000,0.062624,188971.746047,0.070828,174508.470957,[120.50037282004901],"[3.2107577131897194, ..."
1,0.201005,0.060886,189322.168478,0.070589,174553.419507,[159.55987953398085],"[3.0322851096321344, ..."
2,0.402010,0.057590,189986.611775,0.068248,174993.090593,[151.29068554089048],"[2.978635692340847, -..."
3,0.603015,0.057008,190103.928935,0.067707,175094.785447,[150.88729876870724],"[2.7746689554530195, ..."
4,0.804020,0.056578,190190.682519,0.067269,175176.903545,[151.49649189724713],"[2.5001495664227735, ..."


In [17]:
#First Ridge model

ridge_table = reg.run_ridge(X_train, X_test, y_train, y_test)
ridge_table.head()

,alpha,Training_r^2,MSE_train,Testing_r^2,MSE_test,Intercept,Coefficients
1,0.201005,0.062623,188971.958038,0.070876,174499.506415,[401.24823543587866],"[[3.218798001769965, ..."
2,0.402010,0.062621,188972.510009,0.070918,174491.542696,[399.1912779939031],"[[3.22659723784034, -..."
3,0.603015,0.062617,188973.323277,0.070957,174484.386591,[397.1987132647091],"[[3.234129465641445, ..."
4,0.804020,0.062611,188974.350578,0.070991,174477.885311,[395.25144777540675],"[[3.2413911292107387,..."
5,1.005025,0.062605,188975.558980,0.071023,174471.937187,[393.33869754729864],"[[3.2483882916793942,..."


In [14]:
X.columns

Index(['experiences_offered', 'host_is_superhost', 'host_has_profile_pic',
       'host_identity_verified', 'neighbourhood_cleansed', 'is_location_exact',
       'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'bed_type', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'review_scores_rating', 'requires_license', 'instant_bookable',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
       'host_days', 'days_from_last_review', 'amenities_count'],
      dtype='object')